### Introduction:

A friend is looking to open a Indian restaraunt in Toronoto and has asked you to look into what area has the most potential for such a venture. He has described his desire to be in a younger neighborhood with people between 20 - 24 years old. He would also like to know what other Indian restaraunts are in the area to know what his competition will be like. 

### Data
The data used for this project will be provided by Foursquare and https://open.toronto.ca/. 
Current restaraunt data will be gathered and analyzed to see which neighborhood may be a good location for our friend to start looking. Population statistics will be gathered from openToronto. 

### Methodology 
I plan on using the population data first to find out where the younger college age kids live. From this I will narrow down the area that he may want to open a restaurant. Once that area is defined I will search out other Indian restaraunts in that area and group them into smaller groups with k-means square. Once this is done I will make my recommendation to my friend on what area I think fits all of his criteria. I will then find out what other Indian restaraunts are in the area and let him know what the are and where they are located. 


### Results section where you discuss the results.
1. The best location for younger adults living in the city aged 20-24 was determined to be by the Waterfront Communities-The Island and second was the Bay Street Corridor. The close center to both of these neighborhoods was the Union Train station. This then became our center location to serch for indian restaraunts within a 1 km distance. 
2. After searching within the 1km distance we found 11 indian reastaraunts within the defined zone. 
3. We then used k-square means to bunch the restaraunts into smaller neighborhoods to choose a more specific area. 
4. After grouping the restaraunts we came up with a suggested location in cluster 0 that meets all our friends requirements. 
5. We then listed out six Indidan restaraunts in the desired area so he could see what his cometition would look like. 

### Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.
I realized while I was doing this another way of thinking about this problem might involve searching for a neighborhood with an Indian grocery store but few restaraunts and a high amount of youger people. The grocery store would indicate that there are a higher amount of people in that neighborhood who enjoy Inidan food. 

### Conclusion section where you conclude the report.
My conclusion is that he should look to open his restaraunt by the cross streets of King Street and Yonge Street as this neighborhood seems to check all his boxes. 

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [7]:
url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "6e19a90f-971c-46b3-852c-0c48c436d1fc"}
package = requests.get(url, params = params).json()

# Get the data by passing the resource_id to the datastore_search endpoint
# See https://docs.ckan.org/en/latest/maintaining/datastore.html for detailed parameters options
# For example, to retrieve the data content for the first resource in the datastore:

for idx, resource in enumerate(package["result"]["resources"]):
    if resource["datastore_active"]:
        url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/datastore_search"
        p = { "id": resource["id"] }
        data = requests.get(url, params = p).json()
        df = pd.DataFrame(data["result"]["records"])
        break

In [8]:
#Cleaning/Normalizing data 
df.set_index("Category", inplace=True)
df2 = (df.loc[['Population']])

df3 = df2.loc[(df2['Characteristic'] == 'Male: 20 to 24 years' ) | (df2['Characteristic'] == 'Female: 20 to 24 years')]

df3 = df3.drop(['_id','Topic', 'Data Source', 'Characteristic', 'City of Toronto'], axis=1)
df3 = df3.transpose()

df3.columns.values[0] = 'Male population 20-24'
df3.columns.values[1] = 'Female population 20-24'

df3['Male population 20-24'] = df3['Male population 20-24'].astype(int)
df3['Female population 20-24'] = df3['Female population 20-24'].astype(int)

df3['Sum'] = df3['Male population 20-24'] + df3['Female population 20-24']

df3 = df3.sort_values(by='Sum', ascending=False)
df3.head(5)

Category,Male population 20-24,Female population 20-24,Sum
Waterfront Communities-The Island,3100,3590,6690
Bay Street Corridor,2485,3005,5490
Woburn,2310,1970,4280
Willowdale East,2000,2255,4255
Church-Yonge Corridor,1810,2205,4015


In [9]:
CLIENT_ID = '$$$$$$$$$$' # your Foursquare ID
CLIENT_SECRET = '$$$$$$$$$$' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50 #max is 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: $$$$$$$$$$
CLIENT_SECRET:$$$$$$$$$$$$$


In [10]:
address = 'Toronto union Station'
geolocator = Nominatim(user_agent="Foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronoto Union Station is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronoto Union Station is 43.6445345, -79.380381.


In [11]:
search_query = 'Indian Restaurants'
radius = 1000 #In Meters (1 kilometer)
print(search_query + ' .... OK!')

Indian Restaurants .... OK!


In [12]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format( CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=LVLLXEBJRFCCMP0VHHNN4S3GCZ0BWM44DZDGHCQ1GPKD0AGY&client_secret=3Q0VZ2SLZP4E0QIG5BPA05LLIDLJKI2WK3KML30YSPJLLSQV&ll=43.6445345,-79.380381&v=20180604&query=Indian Restaurants&radius=1000&limit=50'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f0e119e40bfaa3334ba9a5f'},
 'response': {'venues': [{'id': '53a07ba3498ee8946e98a7de',
    'name': 'Marigold Indian Bistro | Indian Restaurants in Toronto',
    'location': {'address': '552 Mt Pleasant',
     'lat': 43.64430171166487,
     'lng': -79.39000236526671,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.64430171166487,
       'lng': -79.39000236526671}],
     'distance': 775,
     'postalCode': 'M4S 2M6',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['552 Mt Pleasant', 'Toronto ON M4S 2M6', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d16e941735',
      'name': 'Fast Food Restaurant',
      'pluralName': 'Fast Food Restaurants',
      'shortName': 'Fast Food',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/fastfood_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1594758103',
    'hasPerk': False},
   {

In [14]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
#dataframe

dataframe = dataframe.loc[dataframe['name'].str.contains('Indian')]
dataframe

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,53a07ba3498ee8946e98a7de,Marigold Indian Bistro | Indian Restaurants in...,"[{'id': '4bf58dd8d48988d16e941735', 'name': 'F...",v-1594758103,False,552 Mt Pleasant,43.644302,-79.390002,"[{'label': 'display', 'lat': 43.64430171166487...",775,M4S 2M6,CA,Toronto,ON,Canada,"[552 Mt Pleasant, Toronto ON M4S 2M6, Canada]",NaN,NaN
1,52436962498e0b48fa54e5f8,Indian Palace,[],v-1594758103,False,NaN,43.642756,-79.384285,"[{'label': 'display', 'lat': 43.64275561, 'lng...",371,NaN,CA,NaN,NaN,Canada,[Canada],NaN,NaN
2,50b79b94e4b0a577af25a83f,Indian Roti House,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1594758103,False,256 Queens Quay W,43.639060,-79.385422,"[{'label': 'display', 'lat': 43.63906038875002...",732,M5J 1B5,CA,Toronto,ON,Canada,[256 Queens Quay W (btwn Rees St & Lower Simco...,btwn Rees St & Lower Simcoe S,43722371
3,4fea497c121d2480d3579412,Indian Biriyani House,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1594758103,False,120 Adelaide Street West,43.650050,-79.380662,"[{'label': 'display', 'lat': 43.65005028051833...",614,M5H 3V1,CA,Toronto,ON,Canada,"[120 Adelaide Street West (at Bay Street), Tor...",at Bay Street,NaN
4,4bacfbe4f964a52041213be3,Tamarind: The Indian Kitchen,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1594758103,False,161 Bay St #230,43.646859,-79.378707,"[{'label': 'display', 'lat': 43.646859, 'lng':...",291,M5J 2S1,CA,Toronto,ON,Canada,"[161 Bay St #230 (at Front St.), Toronto ON M5...",at Front St.,NaN
5,4ad4c060f964a5206ff720e3,309 Dhaba Indian Excellence,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1594758103,False,309 King Street West,43.646394,-79.390418,"[{'label': 'display', 'lat': 43.64639421226189...",834,M5V 1J5,CA,Toronto,ON,Canada,"[309 King Street West, Toronto ON M5V 1J5, Can...",NaN,NaN
6,4aef8854f964a5201cd921e3,Aroma Fine Indian Restaurant,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1594758103,False,287 King St. W,43.646463,-79.389644,"[{'label': 'display', 'lat': 43.64646252150344...",776,M5V 1J5,CA,Toronto,ON,Canada,"[287 King St. W (at John St.), Toronto ON M5V ...",at John St.,NaN
7,4d137f8c7a8ba1433cb3d169,Touch - Indian Cuisine,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1594758103,False,1 Adelaide St. E,43.649869,-79.378218,"[{'label': 'display', 'lat': 43.64986899674456...",618,NaN,CA,Toronto,ON,Canada,"[1 Adelaide St. E (at Yonge St), Toronto ON, C...",at Yonge St,NaN
8,5b189725d552c7002c0d64c0,Ram's Indian kitchen,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1594758103,False,199 Bay St 6,43.648026,-79.379819,"[{'label': 'display', 'lat': 43.648025841, 'ln...",391,M5L 1E2,CA,Toronto,ON,Canada,"[199 Bay St 6, Toronto ON M5L 1E2, Canada]",NaN,NaN
10,5b18649e17556200391a45b1,Swades Indian Cuisine,"[{'id': '4bf58dd8d48988d10f941735', 'name': 'I...",v-1594758103,False,208 Queens Quay W,43.639915,-79.381061,"[{'label': 'display', 'lat': 43.639915, 'lng':...",517,M5J 2Y5,CA,Toronto,ON,Canada,"[208 Queens Quay W, Toronto ON M5J 2Y5, Canada]",NaN,NaN


In [15]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

In [16]:
#dataframe_filtered = dataframe_filtered.loc[dataframe_filtered['categories'].str.contains('Indian Restaurant')]

#dataframe = dataframe.loc[dataframe['name'].str.contains('Indian')]
dataframe_filtered = dataframe_filtered.loc[dataframe_filtered['categories'] == 'Indian Restaurant']
dataframe_filtered
                            
#dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
2,Indian Roti House,Indian Restaurant,256 Queens Quay W,43.639060,-79.385422,"[{'label': 'display', 'lat': 43.63906038875002...",732,M5J 1B5,CA,Toronto,ON,Canada,[256 Queens Quay W (btwn Rees St & Lower Simco...,btwn Rees St & Lower Simcoe S,50b79b94e4b0a577af25a83f
3,Indian Biriyani House,Indian Restaurant,120 Adelaide Street West,43.650050,-79.380662,"[{'label': 'display', 'lat': 43.65005028051833...",614,M5H 3V1,CA,Toronto,ON,Canada,"[120 Adelaide Street West (at Bay Street), Tor...",at Bay Street,4fea497c121d2480d3579412
4,Tamarind: The Indian Kitchen,Indian Restaurant,161 Bay St #230,43.646859,-79.378707,"[{'label': 'display', 'lat': 43.646859, 'lng':...",291,M5J 2S1,CA,Toronto,ON,Canada,"[161 Bay St #230 (at Front St.), Toronto ON M5...",at Front St.,4bacfbe4f964a52041213be3
5,309 Dhaba Indian Excellence,Indian Restaurant,309 King Street West,43.646394,-79.390418,"[{'label': 'display', 'lat': 43.64639421226189...",834,M5V 1J5,CA,Toronto,ON,Canada,"[309 King Street West, Toronto ON M5V 1J5, Can...",NaN,4ad4c060f964a5206ff720e3
6,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,43.646463,-79.389644,"[{'label': 'display', 'lat': 43.64646252150344...",776,M5V 1J5,CA,Toronto,ON,Canada,"[287 King St. W (at John St.), Toronto ON M5V ...",at John St.,4aef8854f964a5201cd921e3
7,Touch - Indian Cuisine,Indian Restaurant,1 Adelaide St. E,43.649869,-79.378218,"[{'label': 'display', 'lat': 43.64986899674456...",618,NaN,CA,Toronto,ON,Canada,"[1 Adelaide St. E (at Yonge St), Toronto ON, C...",at Yonge St,4d137f8c7a8ba1433cb3d169
8,Ram's Indian kitchen,Indian Restaurant,199 Bay St 6,43.648026,-79.379819,"[{'label': 'display', 'lat': 43.648025841, 'ln...",391,M5L 1E2,CA,Toronto,ON,Canada,"[199 Bay St 6, Toronto ON M5L 1E2, Canada]",NaN,5b189725d552c7002c0d64c0
10,Swades Indian Cuisine,Indian Restaurant,208 Queens Quay W,43.639915,-79.381061,"[{'label': 'display', 'lat': 43.639915, 'lng':...",517,M5J 2Y5,CA,Toronto,ON,Canada,"[208 Queens Quay W, Toronto ON M5J 2Y5, Canada]",NaN,5b18649e17556200391a45b1
11,Chadani Indian Cuisine,Indian Restaurant,51 Colborne Street,43.649153,-79.374814,"[{'label': 'display', 'lat': 43.649153, 'lng':...",682,M5E 0B7,CA,Toronto,ON,Canada,"[51 Colborne Street, Toronto ON M5E 0B7, Canada]",NaN,5c7969857dc9e1002c4e3819
14,Amaya Fine Indian Cuisine,Indian Restaurant,NaN,43.646989,-79.393345,"[{'label': 'display', 'lat': 43.646989, 'lng':...",1079,NaN,CA,NaN,NaN,Canada,[Canada],NaN,547fcf8d498eea18a9c440c5


In [17]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the Toronto City Center

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Toronto Union Station',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Indian restaurants as blue circle markers
for lat, lng, categories, name in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered['categories'], dataframe_filtered['name']): 
    label = '{}, {}'.format(name,categories)
    label = folium.Popup(label, parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [18]:
dataframe_filtered.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
2,Indian Roti House,Indian Restaurant,256 Queens Quay W,43.639060,-79.385422,"[{'label': 'display', 'lat': 43.63906038875002...",732,M5J 1B5,CA,Toronto,ON,Canada,[256 Queens Quay W (btwn Rees St & Lower Simco...,btwn Rees St & Lower Simcoe S,50b79b94e4b0a577af25a83f
3,Indian Biriyani House,Indian Restaurant,120 Adelaide Street West,43.650050,-79.380662,"[{'label': 'display', 'lat': 43.65005028051833...",614,M5H 3V1,CA,Toronto,ON,Canada,"[120 Adelaide Street West (at Bay Street), Tor...",at Bay Street,4fea497c121d2480d3579412
4,Tamarind: The Indian Kitchen,Indian Restaurant,161 Bay St #230,43.646859,-79.378707,"[{'label': 'display', 'lat': 43.646859, 'lng':...",291,M5J 2S1,CA,Toronto,ON,Canada,"[161 Bay St #230 (at Front St.), Toronto ON M5...",at Front St.,4bacfbe4f964a52041213be3
5,309 Dhaba Indian Excellence,Indian Restaurant,309 King Street West,43.646394,-79.390418,"[{'label': 'display', 'lat': 43.64639421226189...",834,M5V 1J5,CA,Toronto,ON,Canada,"[309 King Street West, Toronto ON M5V 1J5, Can...",NaN,4ad4c060f964a5206ff720e3
6,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,43.646463,-79.389644,"[{'label': 'display', 'lat': 43.64646252150344...",776,M5V 1J5,CA,Toronto,ON,Canada,"[287 King St. W (at John St.), Toronto ON M5V ...",at John St.,4aef8854f964a5201cd921e3


In [19]:
# set number of clusters
kclusters = 3

Toronto_grouped_clustering = dataframe_filtered[['lat','lng']]
#df1 = df[['a', 'b']]

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 1, 1, 0, 0, 2, 0, 1], dtype=int32)

In [20]:
# add clustering labels
Toronto_grouped_clustering.insert(0, 'Cluster Labels', kmeans.labels_)

In [21]:
df = Toronto_grouped_clustering.merge(dataframe_filtered, on=['lat','lng'])

#Toronto_grouped_clustering
df

,Cluster Labels,lat,lng,name,categories,address,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,2,43.639060,-79.385422,Indian Roti House,Indian Restaurant,256 Queens Quay W,"[{'label': 'display', 'lat': 43.63906038875002...",732,M5J 1B5,CA,Toronto,ON,Canada,[256 Queens Quay W (btwn Rees St & Lower Simco...,btwn Rees St & Lower Simcoe S,50b79b94e4b0a577af25a83f
1,0,43.650050,-79.380662,Indian Biriyani House,Indian Restaurant,120 Adelaide Street West,"[{'label': 'display', 'lat': 43.65005028051833...",614,M5H 3V1,CA,Toronto,ON,Canada,"[120 Adelaide Street West (at Bay Street), Tor...",at Bay Street,4fea497c121d2480d3579412
2,0,43.646859,-79.378707,Tamarind: The Indian Kitchen,Indian Restaurant,161 Bay St #230,"[{'label': 'display', 'lat': 43.646859, 'lng':...",291,M5J 2S1,CA,Toronto,ON,Canada,"[161 Bay St #230 (at Front St.), Toronto ON M5...",at Front St.,4bacfbe4f964a52041213be3
3,1,43.646394,-79.390418,309 Dhaba Indian Excellence,Indian Restaurant,309 King Street West,"[{'label': 'display', 'lat': 43.64639421226189...",834,M5V 1J5,CA,Toronto,ON,Canada,"[309 King Street West, Toronto ON M5V 1J5, Can...",NaN,4ad4c060f964a5206ff720e3
4,1,43.646463,-79.389644,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,"[{'label': 'display', 'lat': 43.64646252150344...",776,M5V 1J5,CA,Toronto,ON,Canada,"[287 King St. W (at John St.), Toronto ON M5V ...",at John St.,4aef8854f964a5201cd921e3
5,0,43.649869,-79.378218,Touch - Indian Cuisine,Indian Restaurant,1 Adelaide St. E,"[{'label': 'display', 'lat': 43.64986899674456...",618,NaN,CA,Toronto,ON,Canada,"[1 Adelaide St. E (at Yonge St), Toronto ON, C...",at Yonge St,4d137f8c7a8ba1433cb3d169
6,0,43.648026,-79.379819,Ram's Indian kitchen,Indian Restaurant,199 Bay St 6,"[{'label': 'display', 'lat': 43.648025841, 'ln...",391,M5L 1E2,CA,Toronto,ON,Canada,"[199 Bay St 6, Toronto ON M5L 1E2, Canada]",NaN,5b189725d552c7002c0d64c0
7,2,43.639915,-79.381061,Swades Indian Cuisine,Indian Restaurant,208 Queens Quay W,"[{'label': 'display', 'lat': 43.639915, 'lng':...",517,M5J 2Y5,CA,Toronto,ON,Canada,"[208 Queens Quay W, Toronto ON M5J 2Y5, Canada]",NaN,5b18649e17556200391a45b1
8,0,43.649153,-79.374814,Chadani Indian Cuisine,Indian Restaurant,51 Colborne Street,"[{'label': 'display', 'lat': 43.649153, 'lng':...",682,M5E 0B7,CA,Toronto,ON,Canada,"[51 Colborne Street, Toronto ON M5E 0B7, Canada]",NaN,5c7969857dc9e1002c4e3819
9,1,43.646989,-79.393345,Amaya Fine Indian Cuisine,Indian Restaurant,NaN,"[{'label': 'display', 'lat': 43.646989, 'lng':...",1079,NaN,CA,NaN,NaN,Canada,[Canada],NaN,547fcf8d498eea18a9c440c5


In [23]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(df['lat'], df['lng'], df['Cluster Labels']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [47]:
#df.loc[df['Cluster Labels'] == 0, df.columns[[1] + list(range(5, df.shape[1]))]]
#df.head()
df2 = df.loc[(df['Cluster Labels'] == 0)]
df2
#df3 = df2.loc[(df2['Characteristic'] == 'Male: 20 to 24 years' ) | (df2['Characteristic'] == 'Female: 20 to 24 years')]

,Cluster Labels,lat,lng,name,categories,address,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
1,0,43.650050,-79.380662,Indian Biriyani House,Indian Restaurant,120 Adelaide Street West,"[{'label': 'display', 'lat': 43.65005028051833...",614,M5H 3V1,CA,Toronto,ON,Canada,"[120 Adelaide Street West (at Bay Street), Tor...",at Bay Street,4fea497c121d2480d3579412
2,0,43.646859,-79.378707,Tamarind: The Indian Kitchen,Indian Restaurant,161 Bay St #230,"[{'label': 'display', 'lat': 43.646859, 'lng':...",291,M5J 2S1,CA,Toronto,ON,Canada,"[161 Bay St #230 (at Front St.), Toronto ON M5...",at Front St.,4bacfbe4f964a52041213be3
5,0,43.649869,-79.378218,Touch - Indian Cuisine,Indian Restaurant,1 Adelaide St. E,"[{'label': 'display', 'lat': 43.64986899674456...",618,NaN,CA,Toronto,ON,Canada,"[1 Adelaide St. E (at Yonge St), Toronto ON, C...",at Yonge St,4d137f8c7a8ba1433cb3d169
6,0,43.648026,-79.379819,Ram's Indian kitchen,Indian Restaurant,199 Bay St 6,"[{'label': 'display', 'lat': 43.648025841, 'ln...",391,M5L 1E2,CA,Toronto,ON,Canada,"[199 Bay St 6, Toronto ON M5L 1E2, Canada]",NaN,5b189725d552c7002c0d64c0
8,0,43.649153,-79.374814,Chadani Indian Cuisine,Indian Restaurant,51 Colborne Street,"[{'label': 'display', 'lat': 43.649153, 'lng':...",682,M5E 0B7,CA,Toronto,ON,Canada,"[51 Colborne Street, Toronto ON M5E 0B7, Canada]",NaN,5c7969857dc9e1002c4e3819
10,0,43.648559,-79.371816,Bindia Indian Bistro,Indian Restaurant,16 Market Street,"[{'label': 'display', 'lat': 43.64855916613238...",822,NaN,CA,Toronto,ON,Canada,"[16 Market Street, Toronto ON, Canada]",NaN,526c649111d2200c3216dbc2
